In [ ]:
import pandas as pd
import pymc3 as pm
import seaborn as sns
import matplotlib.pyplot as plt
from Clean import Clean
import time

%matplotlib inline

# Data Prep

In [ ]:
n = 1000

df = pd.read_csv('FM_2000-2019.csv')
print(df.shape)
df_all = df[df['gp_all_0_a'] >= 30]
# df = df_all[0:n]
df = df_all[0:-n]
df_star = df_all[-n:]
print(df.shape)
print(df_star.shape)

In [ ]:
games = 30
q = 1

clean = Clean(df,games)
features = clean.get_features(['e-def-rating','e-off-rating','e-pace'],q)
y = clean.get_target(q).values
cols = features.columns
x = features.values
print(x.shape, y.shape)

clean_test = Clean(df_star,games)
features_test = clean_test.get_features(['e-def-rating','e-off-rating','e-pace'],q)
y_star = clean_test.get_target(q).values
cols_test = features_test.columns
x_star = features_test.values
print(x_star.shape, y_star.shape)

# Fit a Gaussian process (PyMC3)

In [ ]:
with pm.Model() as model:
    a = pm.HalfCauchy('a', beta=1)
    # ls = pm.Normal('ls', mu=0, sigma=1, shape=x.shape[1])
    # ls = pm.HalfNormal('ls', sigma=1, shape=x.shape[1])
    ls = pm.Gamma('ls', alpha=4, beta=3, shape=x.shape[1])
    cov = a**2 * pm.gp.cov.ExpQuad(input_dim=x.shape[1], ls=ls)
    # cov = a**2 * pm.gp.cov.Exponential(input_dim=x.shape[1], ls=ls)
    mean = pm.gp.mean.Constant(y.mean())
    gp = pm.gp.Marginal(mean_func=mean, cov_func=cov)

    noise = pm.HalfNormal('noise', sigma=1)
    y_ = gp.marginal_likelihood("y", X=x, y=y, noise=noise)
    
    # gp = pm.gp.MarginalSparse(mean_func=mean, cov_func=cov, approx='FITC')
    # Xu = pm.gp.util.kmeans_inducing_points(50, x)
    # noise = pm.HalfCauchy('noise', beta=1)
    # y_ = gp.marginal_likelihood("y", X=x, Xu=Xu, y=y, noise=noise)

In [ ]:
start = time.time()
# Get trace
with model:
    trace = pm.sample(1000, cores=1, tune=500)
print("runtime= ", (time.time() - start)/60, 'min')

In [ ]:
pm.save_trace(trace=trace, directory='./gp_trace')

In [ ]:
pm.energyplot(trace);

In [ ]:
pm.traceplot(trace);

In [ ]:
with model:
    ppc = pm.sample_posterior_predictive(trace, samples=10000)
    f_pred = gp.conditional('x_pred', x_star)
    f_star = pm.sample_posterior_predictive(trace
                                             , vars=[f_pred]
                                             , samples=10000) 


In [ ]:
samples = f_star['x_pred']
# samples = ppc['y']

print(samples.shape)
print(samples)
# ppc['obs'].mean(axis=1)

In [ ]:
fig, ax = plt.subplots()
sns.distplot(samples.mean(axis=1), label='Posterior predictive means', ax=ax)
ax.axvline(y_star.mean(), ls='--', color='r', label='True mean')
ax.legend(); 

In [ ]:
samples_0 = samples[:,0]

fig, ax = plt.subplots()
ax.hist(samples_0, bins=50);

In [ ]:
print(y_star[0])
print(samples_0.shape)
print(samples_0.mean())
